In [32]:
# imports
import numpy as np
import pandas as pd
from pandas import Series
pd.set_option('max_columns', None)
pd.set_option('display.max_colwidth', None)

import plotly.express as px

In [25]:
# loading in data
weeks = []
for i in range(8):
    weeks.append(pd.read_csv('../nfl-big-data-bowl-2023/week' + str(i + 1) + '.csv'))
tracking = pd.concat(weeks)

players = pd.read_csv('../nfl-big-data-bowl-2023/players.csv')
scouting = pd.read_csv('../nfl-big-data-bowl-2023/pffScoutingData.csv')
plays = pd.read_csv('../nfl-big-data-bowl-2023/plays.csv')
games = pd.read_csv('../nfl-big-data-bowl-2023/games.csv')

In [77]:
# visualizes a single play
# source: https://www.kaggle.com/code/ammarnassanalhajali/nfl-big-data-bowl-2021-animating-players/notebook
def visualize_play(gameId, playId):
    play = tracking.loc[(tracking['gameId'] == gameId) & 
                  (tracking['playId'] == playId)] 
    fig = px.scatter(play, x="x", y="y", animation_frame="time",
               range_x=[-10,110], range_y=[-10,53.3], 
               color='team')
    fig.update_traces(marker={'size': 9})

    fig.update_layout(paper_bgcolor='#29a500', plot_bgcolor='#29a500', font_color='white',
            width = 800,
            height = 600,
            title = "",

            xaxis = dict(
            nticks = 10,
            title = "",
            visible=False
            ),

            yaxis = dict(
            scaleanchor = "x",
            title = "Temp",
            visible=False
            ),
            showlegend= True,
                      annotations=[
           dict(
                x=-5,
                y=12,
                xref="x",
                yref="y",
                text="HOME ENDZONE",
                font=dict(size=16,color="#e9ece7"),
                align='center',
                showarrow=False,
                yanchor='bottom',
                textangle=-90
            ),
            dict(
                x=105,
                y=10,
                xref="x",
                yref="y",
                text="VISITOR ENDZONE",
                font=dict(size=16,color="#e9ece7"),
                align='center',
                showarrow=False,
                yanchor='bottom',
                textangle=90
            )]  
            ,
            legend=dict(
            traceorder="normal",
            font=dict(family="sans-serif",size=12),
            title = "",
            orientation="h",
            yanchor="bottom",
            y=1.00,
            xanchor="center",
            x=0.5
            ),
        )

    fig.add_shape(type="rect", x0=-10, x1=0,  y0=0, y1=53.3,line=dict(color="#c8ddc0",width=3),fillcolor="#217b00" ,layer="below")
    fig.add_shape(type="rect", x0=100, x1=110, y0=0, y1=53.3,line=dict(color="#c8ddc0",width=3),fillcolor="#217b00" ,layer="below")
    for x in range(0, 100, 10):
        fig.add_shape(type="rect", x0=x,   x1=x+10, y0=0, y1=53.3,line=dict(color="#c8ddc0",width=3),fillcolor="#29a500" ,layer="below")
    for x in range(0, 100, 1):
        fig.add_shape(type="line",x0=x, y0=1, x1=x, y1=2,line=dict(color="#c8ddc0",width=2),layer="below")
    for x in range(0, 100, 1):
        fig.add_shape(type="line",x0=x, y0=51.3, x1=x, y1=52.3,line=dict(color="#c8ddc0",width=2),layer="below")

    for x in range(0, 100, 1):
        fig.add_shape(type="line",x0=x, y0=20.0, x1=x, y1=21,line=dict(color="#c8ddc0",width=2),layer="below")
    for x in range(0, 100, 1):
        fig.add_shape(type="line",x0=x, y0=32.3, x1=x, y1=33.3,line=dict(color="#c8ddc0",width=2),layer="below")

    fig.add_trace(go.Scatter(
        x=[2,10,20,30,40,50,60,70,80,90,98], y=[5,5,5,5,5,5,5,5,5,5,5],
        text=["G","1 0","2 0","3 0","4 0","5 0","4 0","3 0","2 0","1 0","G"],
        mode="text",
        textfont=dict(size=20,family="Arial"),
        showlegend=False,
        ))

    fig.add_trace(go.Scatter(
        x=[2,10,20,30,40,50,60,70,80,90,98], y=[48.3,48.3,48.3,48.3,48.3,48.3,48.3,48.3,48.3,48.3,48.3],
        text=["G","1 0","2 0","3 0","4 0","5 0","4 0","3 0","2 0","1 0","G"],
        mode="text",
        textfont=dict(size=20,family="Arial"),
        showlegend=False,
        ))

    fig.show()

In [78]:
visualize_play(2021090900, 1078)

In [79]:
scouting.head(20)

,gameId,playId,nflId,pff_role,pff_positionLinedUp,pff_hit,pff_hurry,pff_sack,pff_beatenByDefender,pff_hitAllowed,pff_hurryAllowed,pff_sackAllowed,pff_nflIdBlockedPlayer,pff_blockType,pff_backFieldBlock
0,2021090900,97,25511,Pass,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021090900,97,35481,Pass Route,TE-L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021090900,97,35634,Pass Route,LWR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021090900,97,39985,Pass Route,HB-R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021090900,97,40151,Pass Block,C,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0
5,2021090900,97,41233,Pass Route,RWR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2021090900,97,41263,Pass Rush,LEO,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2021090900,97,42377,Pass Block,LT,NaN,NaN,NaN,0.0,0.0,1.0,0.0,42403.0,PP,0.0
8,2021090900,97,42403,Pass Rush,ROLB,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2021090900,97,42404,Pass Block,LG,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,PP,0.0
